In [53]:
%matplotlib inline
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict
import scipy.sparse as sp
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import brier_score_loss
import pickle

In [2]:
import pymysql 
pymysql.install_as_MySQLdb()

ImportError: No module named 'pymysql'

In [3]:
conn_string = 'mysql://{user}:{password}@{host}:{port}/'.format(
    user='root', password='dwdstudent2015', 
    host = '34.225.180.235', port=3306)
engine = create_engine(conn_string, encoding='utf8')

engine.execute('USE ml_football')

In [4]:
# all seasons for England
df = pd.read_sql('select Date, HomeTeam, AwayTeam, FTHG - FTAG diff from England', con = engine)

year = []
month = []
results = []
for i in range(df.shape[0]):
    datetime = df['Date'][i].to_pydatetime()
    year.append(datetime.year)
    month.append(datetime.month)
    if df['diff'][i] > 0:
        results.append(1)
    elif df['diff'][i] == 0:
#         results.append(0.5)
        results.append(2)
    else:
        results.append(0)
df['results'] = results
df['year'] = year
df['month'] = month
df.head()

,Date,HomeTeam,AwayTeam,diff,results,year,month
0,2012-08-18,Arsenal,Sunderland,0.0,2,2012,8
1,2012-08-18,Fulham,Norwich,5.0,1,2012,8
2,2012-08-18,Newcastle,Tottenham,1.0,1,2012,8
3,2012-08-18,QPR,Swansea,-5.0,0,2012,8
4,2012-08-18,Reading,Stoke,0.0,2,2012,8


In [9]:
# season 2012 for England
# df1 = df.loc[df['year'] == 2012]
# df2 = df.loc[df['year'] == 2013].loc[df['month'] < 6]
# df_eng12 = pd.concat([df1,df2])
# df_eng12.tail()

df1 = df.loc[df['year'] == 2016].loc[df['month'] > 7]
df2 = df.loc[df['year'] == 2017].loc[df['month'] < 6]
df_eng12 = pd.concat([df1,df2])
df_eng12.head()

,Date,HomeTeam,AwayTeam,diff,results,year,month
1140,2016-08-13,Burnley,Swansea,-1.0,0,2016,8
1141,2016-08-13,Crystal Palace,West Brom,-1.0,0,2016,8
1142,2016-08-13,Everton,Tottenham,0.0,2,2016,8
1143,2016-08-13,Hull,Leicester,1.0,1,2016,8
1144,2016-08-13,Man City,Sunderland,1.0,1,2016,8


In [10]:
# get data
head = 1140
list_file = [(df_eng12['HomeTeam'][i + head], df_eng12['AwayTeam'][i + head]) for i in range(len(df_eng12['HomeTeam']))]

v = DictVectorizer()
    
# discover corpus and vectorize file word frequencies in a single pass
X = v.fit_transform(Counter(f) for f in list_file)
# or, if you have a pre-defined corpus and/or would like to restrict the words you consider
# in your matrix, you can do
y1 = df_eng12['diff']
y2 = df_eng12['results']
# Corpus = ('aaa', 'bbb', 'cccc', 'dddd', 'xyz')
# v.fit([OrderedDict.fromkeys(Corpus, 1)])
# X = v.transform(Counter(f) for f in (File_1, File_2))

# X is a sparse matrix, but you can access the A property to get a dense numpy.ndarray 
# representation
#print(X)
#print(X.A[0])

#change to normal matrix so that we can change the values
X = X.A

# this is the dictionary for Teams
print(v.vocabulary_)

for i in range(df_eng12.shape[0]):
    X[i][v.vocabulary_[df_eng12['AwayTeam'][i + head]]] = -1;

# X is the feature matrix and y is the ground truth vector (for goal difference)

{'West Ham': 19, 'Burnley': 2, 'Liverpool': 8, 'Leicester': 7, 'Man United': 10, 'Everton': 5, 'Man City': 9, 'Chelsea': 3, 'Watford': 17, 'West Brom': 18, 'Tottenham': 16, 'Hull': 6, 'Sunderland': 14, 'Crystal Palace': 4, 'Middlesbrough': 11, 'Stoke': 13, 'Bournemouth': 1, 'Southampton': 12, 'Arsenal': 0, 'Swansea': 15}


In [12]:
def Brier_Score(true_label, HomeWinProb, DrawProb, AwayWinProb):
    
    """
    @true_label: a list of string to represent the true result, {'HW','Draw','AW'}
    @HomeWinProb:  a list of home wining probability
    @DrawProb: a list of draw probability
    @AwayWinProb: a list of away wining probability
    """
    n = len(true_label)
    
    label_transfer={}
    label_transfer[1]=[1,0,0]
    label_transfer[2]=[0,1,0] 
    label_transfer[0]=[0,0,1] 

    HomeWinProb = np.array(HomeWinProb).reshape((n,1))
    DrawProb = np.array(DrawProb).reshape((n,1))
    AwayWinProb = np.array(AwayWinProb).reshape((n,1))
    true_label = np.array(true_label).reshape((n,1))
    
    total_pro= np.concatenate((HomeWinProb,DrawProb,AwayWinProb),axis=1)
    true_label_matri = [ label_transfer[i[0]] for i in true_label]
    
    true_label_matri = np.asarray(true_label_matri)
    
    res= sum(((true_label_matri-total_pro)**2).T)
    
    return res/3

In [121]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def prob_draw_lose(x):
    if x > 1.5:
        return 1
    elif x < -1.5:
        return 0
    else:
        return float(x) / 3 + 0.5

linear_error_list = []
error_rate_list = []
brier_score_list = []

for j in range(5,0,-1): 
    # hold out the last 5 run as test data
    X_train = X[:-10*j,:]
    X_test = pd.DataFrame(X).tail(10*j)[:10]
    y_train = y1[:-10*j]
    y_train_prob = y2[:-10*j]
    y_test = y1.tail(10*j)[:10]
    y_prob = y2.tail(10*j)[:10] # results: {1,0.5,0}

    # build linear regression model
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    
    reg = linear_model.Ridge(alpha =0.01)
    reg.fit(X_train, y_train)
    y_preds = reg.predict(X_test) # diff
    
    # turn the diff prediction to three probabilities
    y_probH = []
    y_probD = []
    y_probA = []
    y_result = []
    for i in range(len(y_preds)):
        y_probH.append(sigmoid(y_preds[i]))
        #print(y_preds[i])
        y_probD.append(prob_draw_lose(y_preds[i])*(1-y_probH[i]))
        #y_probD.append(prob_draw(y_preds[i])*(1-y_probH[i]))
        y_probA.append(1 - y_probH[i] - y_probD[i])
        if max(y_probH[i], y_probD[i], y_probA[i]) == y_probH[i]:
            y_result.append(1)
        elif max(y_probH[i], y_probD[i], y_probA[i]) == y_probD[i]:
            y_result.append(2)
        else:
            y_result.append(0)
            
    #print(y_result)
    #print(list(y_prob))
    linear_error_list.append(sum([1 for i in range(len(y_result)) if y_result[i] != list(y_prob)[i]]) / len(y_result))
    
    # build multinomial logistic regression
    mr = linear_model.LogisticRegression(tol=1e-10, max_iter=1000, solver='sag',multi_class='multinomial')
    mr.fit(X_train,y_train_prob)
    y_preds_mr = mr.predict_proba(X_test)
    y_preds_class = mr.predict(X_test)
    #print(mr.coef_)
    print(list(y_prob))
    print(list(y_preds_class))
    print('')
    #print(y_preds_class != list(y_prob))
    error_rate_list.append(round(float(np.mean(y_preds_class != list(y_prob))),4))
    #print(y_preds_mr)
    
    
    
    #print([list(v.vocabulary_.keys())[list(v.vocabulary_.values()).index(i)] for i in sorted(range(len(lr.coef_)), key=lambda k: lr.coef_[k], reverse =True)])
    print([list(v.vocabulary_.keys())[list(v.vocabulary_.values()).index(i)] for i in sorted(range(len(reg.coef_)), key=lambda k: reg.coef_[k], reverse =True)])
    
    


#     transfer the probabilities to classification problem
#     class_list = [1.0,2.0,0.0]
#     pred_class = [class_list[np.argmax(list(df_prob.iloc[i,:]))] for i in range(df_prob.shape[0])]
#     print(brier_score_loss(y_prob,pred_class))
#     print(np.mean(pred_class == y_prob))
    
    # transfer the predicted probability to specific format
    y_preds_mr_ = np.array([item for sublist in y_preds_mr for item in sublist]).reshape(3,10)
    brier_score_list_mr = []
    brier_score_list_mr.append(Brier_Score(y_prob,y_preds_mr_[0],y_preds_mr_[1],y_preds_mr_[2]))

    brier_score_list.append(round(float(np.mean(brier_score_list_mr)),4))

print(linear_error_list)
print(error_rate_list)
print(brier_score_list)
#print(X_test)
#print(y_test)
#print(y_preds)

[1, 0, 1, 2, 0, 2, 2, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 2, 1, 0]

['Tottenham', 'Chelsea', 'Man City', 'Liverpool', 'Man United', 'Arsenal', 'Everton', 'West Brom', 'Southampton', 'Crystal Palace', 'Bournemouth', 'Stoke', 'Leicester', 'Burnley', 'West Ham', 'Watford', 'Middlesbrough', 'Swansea', 'Hull', 'Sunderland']
[2, 2, 1, 0, 1, 2, 2, 0, 1, 1]
[1, 0, 1, 0, 0, 1, 1, 1, 1, 1]

['Tottenham', 'Chelsea', 'Man City', 'Liverpool', 'Man United', 'Arsenal', 'Everton', 'West Brom', 'Southampton', 'Crystal Palace', 'Bournemouth', 'Burnley', 'Leicester', 'Stoke', 'West Ham', 'Watford', 'Middlesbrough', 'Swansea', 'Hull', 'Sunderland']
[1, 1, 2, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 0, 1, 0, 1, 1, 0]

['Tottenham', 'Chelsea', 'Man City', 'Liverpool', 'Man United', 'Arsenal', 'Everton', 'West Brom', 'Southampton', 'Leicester', 'Bournemouth', 'Burnley', 'Stoke', 'West Ham', 'Crystal Palace', 'Middlesbrough', 'Watford', 'Swansea', 'Hull', 'Sunderland']
[0, 0, 1, 1, 0, 1, 1, 1, 2, 0]
[0, 1, 1, 1, 0, 1, 1,

### extract features: home advantage, team ability(+ if it's home team, - if it's away team).

In [94]:
# # concatenate beta (trained from linear regression model) to data
# with open('eng16_home_ad.pickle', 'rb') as f:
#     eng16_dict = pickle.load(f)
# f.close()
# #print(eng16_dict)
# beta_train = np.zeros((X_train.shape[0], 2))
# for i in range(len(X_train)):
#     idx_h = list(X_train[i]).index(1)
#     idx_a = list(X_train[i]).index(-1)
#     #print(list(v.vocabulary_.keys())[idx_h])
#     beta_train[i][0] = eng16_dict[list(v.vocabulary_.keys())[idx_h]][0]
#     beta_train[i][1] = eng16_dict[list(v.vocabulary_.keys())[idx_a]][0]
# if X_train[0][-1] in [0, 1]:
#     X_train = np.concatenate((X_train, beta_train), axis = 1)
# #print(X_train[0])

# beta_test = np.zeros((X_test.shape[0], 2))
# X_test = np.array(X_test)
# for i in range(len(X_test)):
#     idx_h = list(X_test[i]).index(1)
#     idx_a = list(X_test[i]).index(-1)
#     #print(list(v.vocabulary_.keys())[idx_h])
#     beta_test[i][0] = eng16_dict[list(v.vocabulary_.keys())[idx_h]][0]
#     beta_test[i][1] = eng16_dict[list(v.vocabulary_.keys())[idx_a]][0]
# if X_test[0][-1] in [0, 1]:
#     X_test = np.concatenate((X_test, beta_test), axis = 1)
# #print(X_test[0])

[ 0.          0.          1.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.         -1.          0.          0.
  0.          0.          0.82715257 -0.35459583]
[ 1.          0.          0.          0.          0.         -1.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -0.45550892  0.62517558]


In [120]:
error_rate_list = []
brier_score_list = []
for j in range(5,0,-1): 
    # hold out the last 5 run as test data
    X_train = X[:-10*j,:]
    X_test = pd.DataFrame(X).tail(10*j)[:10]
    y_train = y1[:-10*j]
    y_train_prob = y2[:-10*j]
    y_test = y1.tail(10*j)[:10]
    y_prob = y2.tail(10*j)[:10] # results: {1,0.5,0}
    
    
    # concatenate beta (trained from linear regression model) to data
    with open('eng16_home_ad.pickle', 'rb') as f:
        eng16_dict = pickle.load(f)
    f.close()
    #print(eng16_dict)
    beta_train = np.zeros((X_train.shape[0], 2))
    for i in range(len(X_train)):
        idx_h = list(X_train[i]).index(1)
        idx_a = list(X_train[i]).index(-1)
        #print(list(v.vocabulary_.keys())[idx_h])
        beta_train[i][0] = eng16_dict[list(v.vocabulary_.keys())[idx_h]][5 - j]
        beta_train[i][1] = eng16_dict[list(v.vocabulary_.keys())[idx_a]][5 - j]
    if X_train[0][-1] in [0, 1]:
        X_train = np.concatenate((X_train, beta_train), axis = 1)
    #print(X_train[0])

    beta_test = np.zeros((X_test.shape[0], 2))
    X_test = np.array(X_test)
    for i in range(len(X_test)):
        idx_h = list(X_test[i]).index(1)
        idx_a = list(X_test[i]).index(-1)
        #print(list(v.vocabulary_.keys())[idx_h])
        beta_test[i][0] = eng16_dict[list(v.vocabulary_.keys())[idx_h]][5 - j]
        beta_test[i][1] = eng16_dict[list(v.vocabulary_.keys())[idx_a]][5 - j]
    if X_test[0][-1] in [0, 1]:
        X_test = np.concatenate((X_test, beta_test), axis = 1)
    #print(X_test[0])

    # build multinomial logistic regression
    mr = linear_model.LogisticRegression(tol=1e-10, max_iter=2000, solver='sag',multi_class='multinomial')
    mr.fit(X_train,y_train_prob)
    y_preds_mr = mr.predict_proba(X_test)
    y_preds_class = mr.predict(X_test)
    #print(mr.coef_)
    print(list(y_prob))
    print(list(y_preds_class))
    print('')
    #print(y_preds_class != list(y_prob))
    error_rate_list.append(round(float(np.mean(y_preds_class != list(y_prob))),4))
    #print(y_preds_mr)
    
    
    # transfer the predicted probability to specific format
    y_preds_mr_ = np.array([item for sublist in y_preds_mr for item in sublist]).reshape(3,10)
    brier_score_list_mr = []
    brier_score_list_mr.append(Brier_Score(y_prob,y_preds_mr_[0],y_preds_mr_[1],y_preds_mr_[2]))

    brier_score_list.append(round(float(np.mean(brier_score_list_mr)),4))

print(error_rate_list)
print(brier_score_list)

#print(X_test)
#print(y_test)
#print(y_preds)

[1, 0, 1, 2, 0, 2, 2, 0, 0, 0]
[1, 0, 1, 1, 1, 1, 1, 2, 1, 0]

[2, 2, 1, 0, 1, 2, 2, 0, 1, 1]
[1, 0, 1, 0, 0, 1, 1, 1, 1, 1]

[1, 1, 2, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 0, 1, 0, 1, 1, 0]

[0, 0, 1, 1, 0, 1, 1, 1, 2, 0]
[0, 0, 1, 1, 0, 1, 1, 2, 0, 0]

[1, 0, 1, 0, 2, 1, 1, 0, 1, 0]
[1, 1, 1, 0, 1, 1, 1, 1, 2, 0]

[0.6, 0.6, 0.2, 0.2, 0.4]
[0.2717, 0.3394, 0.2956, 0.3164, 0.343]
